In [1]:
import numpy as np
import scipy
from scipy import signal
import tensorflow as tf

In [17]:
#load dataset: partial version
X_train = np.load('data/00_marmousi_model_partial.npy')

In [18]:
#create model with E/D LSTM
def crea_model(filt_dims):
    inputs = tf.keras.Input(shape=(None, 1))

    v0 = tf.keras.layers.LSTM(filt_dims, return_sequences=True)(inputs)

    pool_0= tf.keras.layers.MaxPool1D(pool_size=8, strides=2, padding='same')(v0)
    v1 = tf.keras.layers.LSTM(filt_dims*2, return_sequences=True)(pool_0)

    pool_1 = tf.keras.layers.MaxPool1D(pool_size=8, strides=2, padding='same')(v1)
    v2 = tf.keras.layers.LSTM(filt_dims*4, return_sequences=True)(pool_1)

    pool_2 = tf.keras.layers.MaxPool1D(pool_size=8, strides=2, padding='same')(v2)
    v3 = tf.keras.layers.LSTM(filt_dims*8, return_sequences=True)(pool_2)
    v3 = tf.keras.layers.LSTM(filt_dims*8, return_sequences=True)(v3)

    v4 = tf.keras.layers.UpSampling1D(size=2)(v3)
    v4 = tf.keras.layers.LSTM(filt_dims*4, return_sequences=True)(v4)

    v4 = tf.keras.layers.UpSampling1D(size=2)(v4)
    v5 = tf.keras.layers.LSTM(filt_dims*2, return_sequences=True)(v4)

    v5 = tf.keras.layers.UpSampling1D(size=2)(v5)
    v6 =tf.keras.layers.LSTM(filt_dims, return_sequences=True)(v5)

    v = tf.keras.layers.Dense(1, activation='linear')(v6)

    model = tf.keras.Model(inputs=[inputs], outputs=[v])
    model.summary()

    model.compile(optimizer= tf.keras.optimizers.Adamax(learning_rate=0.001), loss = 'mse')

    return model

In [20]:
#compile model
model = crea_model(4)
#train model
history = model.fit(X_train, X_train, epochs = 100)
#save model
model.save('model/saved_model')

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None, 1)]         0         
                                                                 
 lstm_32 (LSTM)              (None, None, 4)           96        
                                                                 
 max_pooling1d_12 (MaxPoolin  (None, None, 4)          0         
 g1D)                                                            
                                                                 
 lstm_33 (LSTM)              (None, None, 8)           416       
                                                                 
 max_pooling1d_13 (MaxPoolin  (None, None, 8)          0         
 g1D)                                                            
                                                                 
 lstm_34 (LSTM)              (None, None, 16)          1600